# Análisis K-Means: Congestión en Santiago

## Objetivo
Aplicar el algoritmo K-Means para identificar patrones de congestión vehicular en Santiago.

## Puntos clave:
1. Preparar los datos (escalar)
2. Identificar número óptimo de clústers (elbow, silhouette, gap)
3. Implementar K-Means con k óptimo
4. Visualizar e interpretar resultados

# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

# Librerías para clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

# Configuración de visualización
plt.style.use('default')
sns.set_theme(style='darkgrid', palette='husl')
%matplotlib inline

print('✓ Librerías importadas correctamente')

In [ ]:
# Cargar datos
df = pd.read_csv('../data/congestion-1.csv')
print(f'Datos cargados: {df.shape[0]} filas x {df.shape[1]} columnas')
df.head()

In [ ]:
# Cargar datos
df = pd.read_csv('congestion-1.csv')
print(f'Datos cargados: {df.shape[0]} filas x {df.shape[1]} columnas')
df.head()

## 2. Exploración inicial de datos

In [ ]:
# Información general
print('=== INFORMACIÓN DEL DATASET ===')
print(f'\nDimensiones: {df.shape}')
print(f'\nValores nulos por columna:')
print(df.isnull().sum()[df.isnull().sum() > 0])
print(f'\nTipos de datos:')
print(df.dtypes.value_counts())

In [ ]:
# Estadísticas descriptivas de variables numéricas clave
variables_clave = ['Duration_hrs', 'Length_km', 'Speed_km/h', 'Peak_Time', 'Hora Inicio', 'Hora Fin']
df[variables_clave].describe()

## 3. Preparación de datos

In [ ]:
# Seleccionar variables numéricas para clustering
# Excluimos variables categóricas originales y nos quedamos con las one-hot encoded
variables_numericas = df.select_dtypes(include=[np.number]).columns.tolist()

# Crear dataset para clustering
df_clustering = df[variables_numericas].copy()

# Eliminar filas con valores nulos si existen
df_clustering = df_clustering.dropna()

print(f'Variables seleccionadas para clustering: {len(df_clustering.columns)}')
print(f'Registros válidos: {len(df_clustering)}')
print(f'\nPrimeras variables: {df_clustering.columns[:10].tolist()}')

In [ ]:
# ESCALADO DE DATOS (StandardScaler)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_clustering)

print('✓ Datos escalados correctamente')
print(f'Shape de datos escalados: {df_scaled.shape}')
print(f'\nMedia de variables escaladas (debe ser ~0): {df_scaled.mean():.6f}')
print(f'Desviación estándar (debe ser ~1): {df_scaled.std():.6f}')

## 4. Determinar número óptimo de clusters

### 4.1 Método del Codo (Elbow Method)

In [ ]:
# Método del Codo
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertias.append(kmeans.inertia_)
    print(f'K={k}: Inercia={kmeans.inertia_:.2f}')

# Visualización
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Inercia (WCSS)', fontsize=12)
plt.title('Método del Codo para K-Means', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_range)
plt.tight_layout()
plt.show()

print('\n💡 Buscar el "codo" donde la inercia deja de disminuir significativamente')

### 4.2 Método de Silhouette

In [ ]:
# Método de Silhouette
silhouette_scores = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(df_scaled)
    score = silhouette_score(df_scaled, labels)
    silhouette_scores.append(score)
    print(f'K={k}: Silhouette Score={score:.4f}')

# Visualización
plt.figure(figsize=(10, 6))
plt.plot(K_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Análisis de Silhouette para K-Means', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_range)
plt.tight_layout()
plt.show()

k_optimo_silhouette = K_range[np.argmax(silhouette_scores)]
print(f'\n✓ K óptimo según Silhouette: {k_optimo_silhouette} (Score: {max(silhouette_scores):.4f})')

### 4.3 Método Gap Statistic

In [ ]:
# Implementación del Gap Statistic
def gap_statistic(data, k_range, n_refs=10, random_state=42):
    """
    Calcula el Gap Statistic para determinar el número óptimo de clusters.
    """
    gaps = []
    std_gaps = []
    
    for k in k_range:
        # Clustering en datos reales
        kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10)
        kmeans.fit(data)
        real_dispersion = np.log(kmeans.inertia_)
        
        # Clustering en datos de referencia (random)
        ref_dispersions = []
        for _ in range(n_refs):
            random_data = np.random.uniform(data.min(), data.max(), size=data.shape)
            kmeans_ref = KMeans(n_clusters=k, random_state=random_state, n_init=10)
            kmeans_ref.fit(random_data)
            ref_dispersions.append(np.log(kmeans_ref.inertia_))
        
        # Calcular gap
        gap = np.mean(ref_dispersions) - real_dispersion
        std_gap = np.std(ref_dispersions)
        
        gaps.append(gap)
        std_gaps.append(std_gap)
        
        print(f'K={k}: Gap={gap:.4f} ± {std_gap:.4f}')
    
    return gaps, std_gaps

# Calcular Gap Statistic
print('Calculando Gap Statistic (puede tomar unos minutos)...\n')
gaps, std_gaps = gap_statistic(df_scaled, K_range, n_refs=10)

# Visualización
plt.figure(figsize=(10, 6))
plt.errorbar(K_range, gaps, yerr=std_gaps, fmt='ro-', linewidth=2, markersize=8, capsize=5)
plt.xlabel('Número de Clusters (k)', fontsize=12)
plt.ylabel('Gap Statistic', fontsize=12)
plt.title('Gap Statistic para K-Means', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_range)
plt.tight_layout()
plt.show()

k_optimo_gap = K_range[np.argmax(gaps)]
print(f'\n✓ K óptimo según Gap Statistic: {k_optimo_gap} (Gap: {max(gaps):.4f})')

### 4.4 Resumen de métodos de selección de K

In [ ]:
# Resumen comparativo
print('='*60)
print('RESUMEN: NÚMERO ÓPTIMO DE CLUSTERS')
print('='*60)
print(f'Método del Codo: Revisar gráfico manualmente')
print(f'Método Silhouette: K = {k_optimo_silhouette}')
print(f'Método Gap Statistic: K = {k_optimo_gap}')
print('='*60)

# Seleccionar K óptimo (usaremos el de Silhouette como referencia)
K_OPTIMO = k_optimo_silhouette
print(f'\n✓ K SELECCIONADO PARA EL ANÁLISIS: {K_OPTIMO}')
print('\n💡 Nota: Puedes cambiar K_OPTIMO manualmente si lo consideras necesario')

## 5. Implementar K-Means con K óptimo

In [ ]:
# Guardar resultados
df_resultado.to_csv('../resultados/resultados_kmeans.csv', index=False)
print('✓ Resultados exportados a: ../resultados/resultados_kmeans.csv')

# Guardar centroides
centroides = pd.DataFrame(scaler.inverse_transform(kmeans_final.cluster_centers_), 
                          columns=df_clustering.columns)
centroides.to_csv('../resultados/centroides_clusters.csv', index=False)
print('✓ Centroides exportados a: ../resultados/centroides_clusters.csv')

## 6. Visualización e Interpretación de Resultados

### 6.1 Boxplots de variables clave por cluster

In [ ]:
# Boxplots de variables importantes
variables_analizar = ['Duration_hrs', 'Length_km', 'Speed_km/h', 'Peak_Time']

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, var in enumerate(variables_analizar):
    df_resultado.boxplot(column=var, by='Cluster', ax=axes[i])
    axes[i].set_title(f'Distribución de {var} por Cluster', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('Cluster', fontsize=11)
    axes[i].set_ylabel(var, fontsize=11)
    axes[i].get_figure().suptitle('')  # Remover título automático

plt.tight_layout()
plt.show()

print('💡 Los boxplots muestran cómo se distribuyen las variables en cada cluster')

### 6.2 Estadísticas descriptivas por cluster

In [ ]:
# Estadísticas por cluster
print('ESTADÍSTICAS DESCRIPTIVAS POR CLUSTER')
print('='*80)

for var in variables_analizar:
    print(f'\n{var}:')
    print(df_resultado.groupby('Cluster')[var].describe().round(2))
    print('-'*80)

### 6.3 Visualización geográfica de clusters

In [ ]:
# Scatter plot de ubicación geográfica coloreado por cluster
plt.figure(figsize=(14, 10))
scatter = plt.scatter(df_resultado['Longitud'], 
                      df_resultado['Latitud'], 
                      c=df_resultado['Cluster'], 
                      cmap='viridis', 
                      alpha=0.6, 
                      s=50,
                      edgecolors='black',
                      linewidth=0.5)

plt.colorbar(scatter, label='Cluster')
plt.xlabel('Longitud', fontsize=12)
plt.ylabel('Latitud', fontsize=12)
plt.title('Distribución Geográfica de Clusters de Congestión', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print('💡 Mapa muestra la distribución geográfica de los diferentes clusters de congestión')

### 6.4 Heatmap de características promedio por cluster

In [ ]:
# Heatmap de características promedio
caracteristicas_clave = ['Duration_hrs', 'Length_km', 'Speed_km/h', 'Peak_Time', 'Hora Inicio', 'Hora Fin']
cluster_means = df_resultado.groupby('Cluster')[caracteristicas_clave].mean()

plt.figure(figsize=(12, 8))
sns.heatmap(cluster_means.T, annot=True, fmt='.2f', cmap='YlOrRd', 
            cbar_kws={'label': 'Valor promedio'}, linewidths=0.5)
plt.title('Heatmap de Características Promedio por Cluster', fontsize=14, fontweight='bold')
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Variable', fontsize=12)
plt.tight_layout()
plt.show()

print('💡 El heatmap facilita la identificación de patrones en cada cluster')

### 6.5 Análisis de silhouette por cluster

In [ ]:
# Visualización detallada de silhouette
from matplotlib import cm

fig, ax = plt.subplots(figsize=(12, 8))

# Calcular valores de silhouette para cada muestra
silhouette_vals = silhouette_samples(df_scaled, clusters)

y_lower = 10
for i in range(K_OPTIMO):
    # Valores de silhouette para cluster i
    cluster_silhouette_vals = silhouette_vals[clusters == i]
    cluster_silhouette_vals.sort()
    
    size_cluster_i = cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = cm.nipy_spectral(float(i) / K_OPTIMO)
    ax.fill_betweenx(np.arange(y_lower, y_upper),
                      0, cluster_silhouette_vals,
                      facecolor=color, edgecolor=color, alpha=0.7)
    
    # Etiqueta del cluster
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    
    y_lower = y_upper + 10

ax.set_title('Análisis de Silhouette por Cluster', fontsize=14, fontweight='bold')
ax.set_xlabel('Coeficiente de Silhouette', fontsize=12)
ax.set_ylabel('Cluster', fontsize=12)

# Línea vertical del promedio
ax.axvline(x=silhouette_final, color="red", linestyle="--", 
           label=f'Silhouette promedio: {silhouette_final:.3f}')
ax.legend()
ax.set_yticks([])
plt.tight_layout()
plt.show()

print('💡 Clusters con valores por encima del promedio están bien definidos')

## 7. Interpretación de Clusters

In [ ]:
# Perfil de cada cluster
print('='*80)
print('PERFIL DE CADA CLUSTER')
print('='*80)

for cluster in range(K_OPTIMO):
    print(f'\n--- CLUSTER {cluster} ---')
    cluster_data = df_resultado[df_resultado['Cluster'] == cluster]
    
    print(f'Número de registros: {len(cluster_data)} ({len(cluster_data)/len(df_resultado)*100:.1f}%)')
    print(f'\nCaracterísticas principales:')
    print(f'  - Duración promedio: {cluster_data["Duration_hrs"].mean():.2f} hrs')
    print(f'  - Longitud promedio: {cluster_data["Length_km"].mean():.2f} km')
    print(f'  - Velocidad promedio: {cluster_data["Speed_km/h"].mean():.2f} km/h')
    print(f'  - Peak Time promedio: {cluster_data["Peak_Time"].mean():.2f}')
    
    # Comuna más frecuente
    if 'Commune' in cluster_data.columns:
        comuna_top = cluster_data['Commune'].mode()
        if len(comuna_top) > 0:
            print(f'  - Comuna más frecuente: {comuna_top.values[0]}')
    
    print('-'*80)

## 8. Exportar resultados

In [ ]:
# Guardar resultados
df_resultado.to_csv('resultados_kmeans.csv', index=False)
print('✓ Resultados exportados a: resultados_kmeans.csv')

# Guardar centroides
centroides = pd.DataFrame(scaler.inverse_transform(kmeans_final.cluster_centers_), 
                          columns=df_clustering.columns)
centroides.to_csv('centroides_clusters.csv', index=False)
print('✓ Centroides exportados a: centroides_clusters.csv')

## 9. Conclusiones

### Resumen del análisis:

1. **Preparación de datos**: Se escalaron todas las variables numéricas usando StandardScaler

2. **Número óptimo de clusters**: Se determinó usando tres métodos:
   - Método del Codo (Elbow)
   - Silhouette Score
   - Gap Statistic

3. **Implementación**: Se aplicó K-Means con el número óptimo de clusters identificado

4. **Visualización**: Se generaron múltiples gráficos para interpretar los resultados:
   - Boxplots de variables clave
   - Distribución geográfica
   - Heatmap de características
   - Análisis de silhouette

### Interpretación:

Los clusters identificados representan diferentes patrones de congestión vehicular en Santiago, diferenciados por:
- Duración de la congestión
- Longitud del segmento congestionado
- Velocidad promedio
- Hora del día
- Ubicación geográfica

Esta segmentación permite identificar zonas y horarios críticos de congestión para implementar medidas de mitigación específicas.